### Import Python packages

In [422]:
import pandas as pd #Python data analysis package
import numpy as np #Fundamental package for scientific computing in Python
from pyDOE2 import * #Experimental design package for Python

### Process user input

In [423]:
# Define csv input file
input_file = 'proCFPS_parameters.csv'

# Import csv input file into a dataframe
input = pd.read_csv(input_file)

### Recover fixed parameters, variable parameters and maximum concentrations

In [424]:
for status in input.columns:
    
    if (input[status] == 'fixed').any():
        fixed_parameters = input[input['Status'] == 'fixed']
        maximum_fixed_concentrations = fixed_parameters['Maximum concentration'].tolist()
        fixed_parameters = fixed_parameters['Parameter'].to_numpy()
        n_fixed_parameters = np.shape(fixed_parameters)[0]
    
    if (input[status] == 'variable').any():
        variable_parameters = input[input['Status'] == 'variable']
        maximum_variable_concentrations = variable_parameters['Maximum concentration'].to_numpy()
        maximum_variable_concentrations = np.reshape(maximum_variable_concentrations, (1, n_variable_parameters))
        variable_parameters = variable_parameters['Parameter'].to_numpy()
        n_variable_parameters = np.shape(variable_parameters)[0]

### Latin hypercube sampling

In [425]:
# Define number of parameters to be sampled
n_parameters = n_variable_parameters

# Define number of samples to generate
n_samples = 100

# Define number of parameters' concentration ratios
n_ratios = 5

# Perform latin hypercube sampling
sampling = lhs(n_parameters, samples=n_samples)

In [426]:
# Define levels
levels = (1 / n_ratios)

level_1 = levels*0
level_2 = levels*1
level_3 = levels*3
level_4 = levels*4
level_5 = levels*5

# Generate multi-level list 
levels_list = []

for sample in sampling:
    new_sample = []
    
    for parameter in sample:

        if parameter >= level_1 and parameter < level_2:
            new_sample.append(level_1)
            continue
        
        if parameter >= level_2 and parameter < level_3:
            new_sample.append(level_2)
            continue
        
        if parameter >= level_3 and parameter < level_4:
            new_sample.append(level_3)
            continue
        
        if parameter >= level_4 and parameter < level_5:
            new_sample.append(level_4)
            continue
            
        if parameter >= level_5:
            new_sample.append(level_5)
            continue

    levels_list.append(new_sample)

# Convert multi-level list to a multi-level array
levels_array = np.asarray(levels_list)

### Generate concentrations array for variable parameters

In [427]:
# Multiply ratios array by maximum concentrations vector of variable parameters
variable_concentrations_array = np.multiply(levels_array, maximum_variable_concentrations)
type(maximum_fixed_concentration)

float

### Generate concentrations array for fixed parameters

In [428]:
nrows_variable_concentrations_array = np.shape(variable_concentrations_array)[0]
fixed_concentrations_array_list = []

for maximum_fixed_concentration in maximum_fixed_concentrations:
    fixed_concentrations_array = np.full(nrows_variable_concentrations_array, maximum_fixed_concentration)
    fixed_concentrations_array_list.append(fixed_concentrations_array)

fixed_concentrations_array = np.stack(fixed_concentrations_array_list, axis=-1)

### Define control samples

In [429]:
# Define control sample 1 (Maximum concentration): all parameters at maximum concentration
maximum_concentrations_dict = dict(input[['Parameter', 'Maximum concentration']].to_numpy())
maximum_concentrations_sample = np.fromiter(maximum_concentrations_dict.values(), dtype=float)

# Define control sample 2 (Autolfuorescence): control sample 1 minus DNA 
autofluorescence_dict = maximum_concentrations_dict

if 'GOI-DNA' in autofluorescence_dict:
    autofluorescence_dict['GOI-DNA']=0

if 'GFP-DNA' in autofluorescence_dict:
    autofluorescence_dict['GFP-DNA']=0

autofluorescence_sample = np.fromiter(autofluorescence_dict.values(), dtype=float)

# Generate one single control array
nrows_autofluorescence_sample = np.shape(autofluorescence_sample)[0]
autofluorescence_sample = np.reshape(autofluorescence_sample, (1, nrows_autofluorescence_sample))

nrows_maximum_concentrations_sample = np.shape(maximum_concentrations_sample)[0]
maximum_concentrations_sample = np.reshape(maximum_concentrations_sample, (1, nrows_maximum_concentrations_sample))

control_concentrations_array = np.concatenate((maximum_concentrations_sample, autofluorescence_sample), axis=0)

### Generate initial concentration plate combinations

In [430]:
default_concentrations_array = np.concatenate((variable_concentrations_array, fixed_concentrations_array), axis=1)
full_concentrations_array = np.concatenate((default_concentrations_array, control_concentrations_array), axis=0)
full_concentrations_df = pd.DataFrame(full_concentrations_array)

### Saving initial concentration plate combinations into a csv file

In [431]:
all_parameters = input['Parameter'].tolist()
output = full_concentrations_df.to_csv('initial_plate.csv', header=all_parameters)